## Exercise to try to know who wrote a song

In [1]:
"""
Bag of Words:
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
"""

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from os import listdir
from os.path import isfile, join
import requests
import re
import numpy as np

RATIO_TEST_DATA = 0.2

## Reading the file and cleaning the text

In [ ]:
'''save data in files'''
html = {}
lyrics = {}

singers= ['eminem','madonna']
for i in singers:
    lyrics.update({i:[]})
    files = listdir('songtexte/'+i)
    
    for file in files[:30]:
        out = open('songtexte/'+i+'/'+file)
        html.update({file:out.read()})
            

    ''' cleanind html'''
    
    for z in html:
        pattern = '<!-- Usage of azlyrics.com .* agreement. Sorry about that. -->([\w\W]+)<!-- MxM banner -->'
        text = re.findall(pattern,html[z])
        if text:
        	lyrics[i].append(text[0].replace('<br>','').replace('</div>','').replace('<i>','').replace('</i>','')) 
		


## split the data for each  singer, and create a Y variable of the same lenght

In [ ]:
''' start of the prediction'''


eminem = lyrics['eminem']
madonna = lyrics['madonna']

print("Madonna: {} songs".format(len(madonna)))
print("Eminem : {} songs".format(len(eminem)))

X = madonna + eminem
y = ['madonna'] * len(madonna) + ['eminem'] * len(eminem)


# split into training and test set
Xtrain, Xtest, ytrain, ytest = \
    model_selection.train_test_split(X, y, test_size=RATIO_TEST_DATA, random_state=42)


## definition and fitting of the model

In [ ]:

#
# prediction pipeline
#

# smoothing parameter
# try 1.0 and smaller in steps of factor 10
ALPHA = 0.1

# words must appear at least MIN_DF times
MIN_DF=10

model = Pipeline([
        ('vectorizer', CountVectorizer(min_df=MIN_DF, ngram_range=(1, 1))),
        ('tfidf_transformer', TfidfTransformer()),
        ('bayes_model', MultinomialNB(alpha=ALPHA)),
])

# train the model
model.fit(Xtrain, ytrain)


## Evaluation of the model

In [ ]:
vect = model.named_steps['vectorizer']
nwoerter = len(vect.vocabulary_)
print("\nnumber of words: {}".format(nwoerter))

# evaluate the model quality
print("\naccuracy on training data: ", model.score(Xtrain, ytrain))
print("accuracy on test data: ", model.score(Xtest, ytest))


# 10-fold cross-validation
print("\nCross-validation:")
print(model_selection.cross_val_score(model, X, y, cv=10, scoring='accuracy'))


# print distinct words
names = np.array(model.named_steps['vectorizer'].get_feature_names())
coef = model.named_steps['bayes_model'].coef_
coef = coef.reshape((len(names),))

# top words for Eminem (-coef for Madonna)
indices = (coef).argsort()[:20].tolist()
print("\ndistinguishing words for Eminem:")
print(names[indices])


## test of a predictions

In [ ]:

#
# Make a prediction
#
examples = [
    "we will dance and have a good time", 
    "my homies are nuts and listen to dumb records",
    "like a dsfjklweh"
]

predictions = model.predict(examples)

for satz, vorh in zip(examples, predictions):
    print("\nThe phrase '{}' is by '{}'".format(satz, vorh))



## Creation of a website with bottle to know who wrote each song

In [ ]:
'''bottle '''



from bottle import default_app, route, run
from bottle import get, post, request


@route('/')
def index():
    
    cast = "<h3>dramatis personae:</h3>"
    for char in singers:
        cast += '<li><a href="/{0}">{0}</a></li>'.format(char)
    return '<ul>{}</ul>'.format(cast)

@route('/<char>')
def enter_char(char):
    links = ''
    for i in lyrics[char]:
        links +=  '''<p>{}'</p>'''.format(i,i)
    html = '''<h1>Enter {}</h1>{}'''.format(char,links) 
    return html

@get('/form')
def form():
    return '''
        <form action="/form" method="post">
            Lyrics: <input name="lyrics" type="text" />
            <input value="Login" type="submit" />
        </form>
    '''

@post('/form')
def eval_form():
    lyrics = request.forms.get('lyrics')
    predictions = model.predict([lyrics])
    return ("\nThe phrase '{}' is by '{}'".format(lyrics, predictions[0]))
    


application = default_app()
run(application, host='localhost', port=8080, reloader=True)